# Evaluation

> Functions for evaluating LLaVA models, including prediction generation and metric calculation.

In [ ]:
#| default_exp evaluation

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import sys
from pathlib import Path
import os
import gc
import json
import torch
import time
from tqdm.auto import tqdm
from typing import Union, Optional, Dict, Any, List, Type # Added Dict, Any, List, Type
import wandb # Added wandb import
import argparse
import PIL.Image # Added PIL

from fastai.learner import Learner
from fastai.data.load import DataLoader
from transformers import AutoTokenizer
from transformers.modeling_outputs import CausalLMOutputWithPast

# Add dependency for Levenshtein distance calculation
try:
    import Levenshtein
except ImportError:
    print("Warning: `pip install python-Levenshtein` is required for ANLS evaluation.")
    Levenshtein = None

# --- Project Root Setup --- 
project_root = Path(os.getcwd())
# Check if running from nbs/
if project_root.name == 'nbs' and (project_root.parent / 'settings.ini').exists():
    project_root = project_root.parent
# Check if running from scripts/
elif project_root.name == 'scripts' and (project_root.parent / 'settings.ini').exists():
     project_root = project_root.parent
elif not (project_root / 'settings.ini').exists():
     # Try going up one level if settings.ini not found directly
     if (project_root.parent / 'settings.ini').exists():
          project_root = project_root.parent
     else:
          print("Warning: Could not automatically determine project root. Assuming current dir.")
          # Fallback: Assume running from project root if structure unknown

project_root_str = str(project_root.resolve())
if project_root_str not in sys.path:
    print(f"Adding project root to sys.path: {project_root_str}")
    sys.path.insert(0, project_root_str)
else:
    # print(f"Project root already in sys.path: {project_root_str}") # Less verbose
    pass
# --- End Project Root Setup --- 
    
# Import necessary llava components
try:
    from llava.utils import load_config, init_wandb # Added init_wandb
    from llava.data.loading import get_test_dataloader, LLaVADataBlockStage2, get_image_path, LLaVASample # Assumes this function exists in 10_data_loading
    from llava.model.baseline import BaselineLLaVAModel 
    from llava.model.adaptive import AdaptiveLLaVAModel # Import adaptive model
    from llava.data.preprocessing import tokenizer, DEFAULT_IMAGE_TOKEN, IGNORE_INDEX, IMAGE_TOKEN_INDEX_PLACEHOLDER # Import tokenizer & constants
    # Import PEFT related things if needed for model loading check
    try:
        from peft import PeftModel
        _peft_available = True
    except ImportError:
        PeftModel = None 
        _peft_available = False
except ImportError as e:
    print(f"Error importing llava modules: {e}. Make sure nbdev_export was run.")
    # Define placeholders if running standalone or during initial setup
    def load_config(path): return {}
    def init_wandb(*args, **kwargs): pass
    def get_test_dataloader(config, dataset_name, dblock=None): raise NotImplementedError
    class BaselineLLaVAModel(torch.nn.Module): 
        def __init__(self, *args, **kwargs): 
            super().__init__()
            self.dummy = torch.nn.Linear(1,1)
            self.image_token_index_marker = -200 # Define necessary attributes
            self.projector = torch.nn.Linear(10,10)
            self.language_model = self # Make model act like LLM for generate
            self.vision_tower = self # Dummy vision tower
            self.config = {'model':{}} # Dummy config
        def encode_image(self, *args, **kwargs): return torch.randn(1,5,10) # Dummy image features B, P, D
        def get_input_embeddings(self): return torch.nn.Embedding(100, 10)
        def forward(self, *args, **kwargs): return CausalLMOutputWithPast(logits=torch.randn(1, 10, 100))
        def generate(self, *args, **kwargs): return torch.randint(0, 100, (1, kwargs.get('max_new_tokens', 10)))
        def to(self, *args, **kwargs): return self # Avoid device errors in dummy
        def eval(self): pass
    class AdaptiveLLaVAModel(BaselineLLaVAModel): pass # Simple inheritance for placeholder
    class DummyTokenizer:
         def __init__(self): self.eos_token_id=1; self.pad_token_id=0; self.unk_token_id=2
         def decode(self, *args, **kwargs): return "dummy decoded text"
         def convert_tokens_to_ids(self, *args): return self.unk_token_id
         def __len__(self): return 100
    tokenizer = DummyTokenizer()
    DEFAULT_IMAGE_TOKEN = "<image>"
    IGNORE_INDEX = -100
    IMAGE_TOKEN_INDEX_PLACEHOLDER = -200
    LLaVADataBlockStage2 = None # Placeholder
    PeftModel = None
    _peft_available = False

Adding project root to sys.path: /workspace/llava


## Step 5.2: Prediction Generation

This function takes a trained fastai `Learner` and a test `DataLoader`, generates predictions, decodes them into text, and saves them to a JSON file. It now also includes efficiency logging.

In [ ]:
#| export

def get_raw_images_for_batch(dl: DataLoader, batch_indices: List[int]) -> List[Optional[PIL.Image.Image]]:
    """Safely retrieves the raw PIL images for a given batch of indices from the DataLoader's dataset.

    Args:
        dl: The DataLoader instance.
        batch_indices: A list of integer indices corresponding to the items in the current batch.

    Returns:
        A list of PIL.Image.Image objects or None for each item in the batch.
    """
    raw_images = []
    if not hasattr(dl, 'dataset'):
        print("Warning: DataLoader has no 'dataset' attribute. Cannot retrieve raw images.")
        return [None] * len(batch_indices)

    # Try to get original items from dataloader if available
    original_items = getattr(dl, 'items', None)
    if original_items is None:
        print("Warning: DataLoader has no 'items' attribute. Cannot reliably retrieve raw image paths.")
        return [None] * len(batch_indices)

    for idx in batch_indices:
        try:
            if idx < len(original_items):
                original_item = original_items[idx]
                if isinstance(original_item, LLaVASample):
                    image_path = get_image_path(original_item)
                    if image_path.exists():
                        raw_images.append(PIL.Image.open(image_path))
                    else:
                        print(f"Warning: Image file not found at {image_path} for index {idx}")
                        raw_images.append(None)
                else:
                    # Attempt to infer path if item is just a path or tuple
                    potential_path = None
                    if isinstance(original_item, (str, Path)):
                         potential_path = Path(original_item)
                    elif isinstance(original_item, tuple) and len(original_item)>0 and isinstance(original_item[0], (str, Path)):
                         potential_path = Path(original_item[0])
                    
                    if potential_path and potential_path.exists():
                        raw_images.append(PIL.Image.open(potential_path))
                    else:
                         print(f"Warning: Could not determine image path from item at index {idx}: {original_item}")
                         raw_images.append(None) # Cannot determine path from item format
            else:
                 print(f"Warning: Index {idx} out of bounds for dl.items (length {len(original_items)})")
                 raw_images.append(None)

        except Exception as e:
            print(f"Warning: Could not load raw image for dataset index {idx}: {e}")
            raw_images.append(None)
    return raw_images


def generate_predictions(learner: Learner, 
                         test_dl: DataLoader, 
                         output_file: Union[str, Path], 
                         max_len: int = 200, # Max generation length
                         temperature: float = 0.2, # Generation temperature
                         top_p: Optional[float] = None, # Nucleus sampling top_p
                        ):
    """Generates predictions for a test dataloader and saves them to a JSON Lines file.
    Includes efficiency logging (latency, peak VRAM).

    Uses the HF generate method on the underlying language model component,
    manually preparing the combined image and text embeddings.
    Handles passing `raw_images` if the model is adaptive.

    Args:
        learner: Trained fastai Learner object containing the model.
                 Expected to have `learner.model`, `learner.dls.device`,
                 and potentially `learner.tokenizer` or `learner.dls.tokenizer`.
        test_dl: DataLoader for the test set.
        output_file: Path to save the JSON Lines prediction file.
        max_len: Maximum number of new tokens to generate.
        temperature: Sampling temperature for generation. Set to 0 for greedy decoding.
        top_p: If set (and temperature > 0), use nucleus sampling with this top_p value.
    """
    output_file = Path(output_file)
    output_file.parent.mkdir(parents=True, exist_ok=True)
    
    model = learner.model
    is_adaptive = isinstance(model, AdaptiveLLaVAModel)
    
    # Get tokenizer
    tok = getattr(learner, 'tokenizer', getattr(learner.dls, 'tokenizer', tokenizer))
    if tok is None:
        raise AttributeError("Tokenizer not found. Cannot decode predictions.")
        
    # Ensure model components exist
    if not all(hasattr(model, attr) and getattr(model, attr) is not None 
               for attr in ['vision_tower', 'projector', 'language_model', 'get_input_embeddings', 'encode_image']):
        raise AttributeError("Model is missing required components.")
        
    model.eval() # Set model to evaluation mode
    results = []
    total_time = 0
    num_samples = 0
    device = learner.dls.device # Get device from learner

    print(f"Generating predictions for {len(test_dl.dataset)} samples...")
    print(f"Saving predictions to: {output_file}")
    
    # Reset CUDA memory stats
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats(device=device) 
        print("Reset CUDA peak memory stats before generation.")

    with torch.no_grad(), open(output_file, 'w') as f_out:
        batch_count = 0
        for batch_tuple in tqdm(test_dl, desc="Generating Predictions"):
            start_time = time.time() # Start timing
            
            # The dataloader yields tuples (or dicts if custom collate). Assume dict for now.
            # If it yields tuples, need to adjust access.
            batch = batch_tuple # Assuming the batch is already a dict
            if not isinstance(batch, dict):
                 # Fallback if it's a tuple (e.g., (images, texts))
                 # This assumes standard fastai dataloader output without custom collate
                 print(f"Warning: Batch type is {type(batch)}, expected dict. Trying tuple access.")
                 try:
                      # Need to manually apply the batch transform logic here if not done by DL
                      # This is complex. It's better to ensure the DL yields the dict.
                      # For now, assume batch *is* the dict for simplicity
                      raise NotImplementedError("DataLoader did not yield a dictionary batch. Ensure LLaVABatchTransform is used.")
                 except (TypeError, IndexError, NotImplementedError) as e:
                      print(f"Error processing non-dict batch: {e}. Skipping batch.")
                      continue
                 
            pixel_values = batch['pixel_values'].to(device)
            input_ids = batch['input_ids'].to(device) 
            batch_size = pixel_values.shape[0]
            
            # --- Retrieve sample IDs and raw images --- 
            # Use fastai's internal batch indexing if available
            start_idx = batch_count * test_dl.bs
            end_idx = start_idx + batch_size
            current_indices = test_dl.idxs[start_idx:end_idx] if hasattr(test_dl, 'idxs') else list(range(num_samples, num_samples + batch_size))
            batch_ids = []
            if hasattr(test_dl, 'items') and test_dl.items is not None:
                 for idx in current_indices:
                      item_id = f"sample_{idx}"
                      try:
                          original_item = test_dl.items[idx]
                          if isinstance(original_item, LLaVASample):
                              item_id = original_item.sample_id
                          elif isinstance(original_item, dict) and 'id' in original_item:
                              item_id = original_item['id']
                      except IndexError:
                           pass # Keep default ID if index out of bounds
                      except Exception as e:
                           print(f"Warning: Error getting ID for index {idx}: {e}")
                      batch_ids.append(item_id)
            else:
                 batch_ids = [f"sample_{idx}" for idx in current_indices]
                 
            raw_images = None
            if is_adaptive:
                raw_images = get_raw_images_for_batch(test_dl, current_indices)
                if len(raw_images) != batch_size:
                    print(f"Warning: Mismatch between raw_images ({len(raw_images)}) and batch_size ({batch_size}). Generation might fail.")
                    raw_images = [None] * batch_size # Pad with None to avoid errors
            # ------------------------------------------ 

            # --- Generation --- 
            image_features = model.encode_image(pixel_values) # (B, P, D_clip)
            if image_features is None: 
                print("Warning: Image encoding failed for batch. Skipping.")
                num_samples += batch_size
                total_time += (time.time() - start_time)
                batch_count += 1
                continue 
            projected_features = model.projector(image_features) # (B, P, D_llm)
            
            outputs_list = []
            for i in range(batch_size):
                current_input_ids = input_ids[i:i+1] 
                current_proj_features = projected_features[i:i+1]
                
                marker = getattr(model, 'image_token_index_marker', IMAGE_TOKEN_INDEX_PLACEHOLDER)
                image_token_indices = torch.where(current_input_ids[0] == marker)[0]
                
                if len(image_token_indices) == 0:
                    print(f"Warning: Image marker {marker} not found in sample {batch_ids[i]}. Skipping.")
                    outputs_list.append(torch.tensor([tok.eos_token_id], device=device)) 
                    continue
                
                image_token_start_index = image_token_indices[0].item()
                input_ids_no_marker = current_input_ids.clone()
                input_ids_no_marker[input_ids_no_marker == marker] = 0 
                text_embeddings = model.get_input_embeddings()(input_ids_no_marker)
                
                text_emb_before = text_embeddings[:, :image_token_start_index]
                text_emb_after = text_embeddings[:, image_token_start_index + 1:]
                
                prompt_embeds = torch.cat([
                    text_emb_before,
                    current_proj_features.to(text_embeddings.device, dtype=text_embeddings.dtype),
                    text_emb_after
                ], dim=1)
                
                llm_component = model.language_model
                if _peft_available and isinstance(llm_component, PeftModel):
                     llm_component = llm_component.base_model.model 
                
                prompt_attention_mask = torch.ones(prompt_embeds.shape[:2], dtype=torch.long, device=device)
                
                try:
                    gen_params = {
                        "inputs_embeds": prompt_embeds,
                        "attention_mask": prompt_attention_mask,
                        "max_new_tokens": max_len,
                        "eos_token_id": tok.eos_token_id,
                        "pad_token_id": tok.pad_token_id if tok.pad_token_id is not None else tok.eos_token_id,
                        "do_sample": (temperature > 0),
                        "num_beams": 1
                    }
                    if temperature > 0:
                        gen_params["temperature"] = temperature
                        if top_p is not None: gen_params["top_p"] = top_p
                    else: 
                        gen_params["temperature"] = 1.0 
                        gen_params["do_sample"] = False
                        
                    # NOTE: Passing raw_images to generate is not directly supported by HF's standard generate.
                    # If the AdaptivePatcher needs raw_image during the forward pass *called by generate*,
                    # this requires a custom generation loop or modification of the model's generate method.
                    # Our VariableResolutionPatcher determines strategy *before* forward, so it's currently okay.
                    
                    output_ids_gen = llm_component.generate(**gen_params)
                    
                    # Remove the prompt tokens from the generated output
                    output_ids_gen = output_ids_gen[:, prompt_embeds.shape[1]:]
                    outputs_list.append(output_ids_gen[0]) 
                except Exception as gen_e:
                     print(f"Error during generation for sample {batch_ids[i]}: {gen_e}")
                     outputs_list.append(torch.tensor([tok.eos_token_id], device=device)) # Fallback
            
            # --- Decode and Save --- 
            end_time = time.time() 
            total_time += (end_time - start_time)
            num_samples_in_batch = batch_size

            for i, gen_ids in enumerate(outputs_list):
                decoded_text = tok.decode(gen_ids.cpu(), skip_special_tokens=True).strip()
                result_entry = {"id": batch_ids[i], "prediction": decoded_text}
                f_out.write(json.dumps(result_entry) + '\n')
                results.append(result_entry)
            
            num_samples += num_samples_in_batch
            batch_count += 1

    # --- Log Efficiency Metrics --- 
    avg_latency = (total_time / num_samples) * 1000 if num_samples > 0 else 0
    print(f"Finished generation. Saved {len(results)} predictions to {output_file}")
    print(f"Average inference latency: {avg_latency:.2f} ms/sample")
    
    if wandb.run is not None:
        wandb.log({"eval/avg_inference_latency_ms": avg_latency})

    if torch.cuda.is_available():
        peak_vram_gb = torch.cuda.max_memory_allocated(device=device) / (1024**3) 
        print(f"Peak Inference VRAM used: {peak_vram_gb:.2f} GB")
        if wandb.run is not None:
            wandb.log({"eval/peak_inference_vram_gb": peak_vram_gb})
        torch.cuda.reset_peak_memory_stats(device=device) 
        
    return results

## Step 5.3 & 5.5: Integrate External Evaluation Scripts (Implement VQAv2)

Define functions for calling external evaluation scripts. The `evaluate_vqa` function is now implemented using the official VQA tools.

In [ ]:
#| export
def evaluate_vqa(preds_file: Union[str, Path], 
                 gt_file: Union[str, Path], 
                 ques_file: Optional[Union[str, Path]] = None, # Path to questions file if needed by API
                 **kwargs):
    """Evaluates VQAv2 predictions using the official VQA evaluation tools.
    
    Args:
        preds_file: Path to the JSON Lines prediction file generated by `generate_predictions`
                    (expected format: {'id': question_id, 'prediction': answer}).
        gt_file: Path to the ground truth annotation file (e.g., v2_mscoco_val2014_annotations.json).
        ques_file: Path to the questions file (e.g., v2_OpenEnded_mscoco_val2014_questions.json). 
                   Required by the standard VQA eval tools.
        **kwargs: Additional arguments (unused currently).
    
    Returns:
        Dictionary containing VQA scores (e.g., {'overall': score, 'yes/no': ..., 'number': ..., 'other': ...}), 
        or {'overall': 0.0} if evaluation fails.
    """
    print(f"--- VQA Evaluation --- ")
    print(f"Predictions file: {preds_file}")
    print(f"Ground Truth Annotation file: {gt_file}")
    print(f"Ground Truth Questions file: {ques_file}")
    results = {"overall": 0.0} # Default return value
    
    if ques_file is None:
        print("Error: Path to questions file (ques_file) is required for VQA evaluation.")
        return results

    try:
        # --- Load VQA Tools --- 
        # Assumes the VQA evaluation tools directory ('PythonHelperTools') is in the Python path.
        # User needs to download from https://visualqa.org/download.html
        # and add the PythonHelperTools directory to their PYTHONPATH or sys.path
        try:
            from vqaTools.vqa import VQA
            from vqaEvaluation.vqaEval import VQAEval
        except ImportError:
            print("Error: VQA evaluation tools (vqaTools/vqaEvaluation) not found in PYTHONPATH.")
            print("Please download from https://visualqa.org/download.html and add PythonHelperTools to your path.")
            return results
        
        # --- Load and Reformat Predictions --- 
        vqa_preds_formatted = []
        print(f"Loading and reformatting predictions from {preds_file}...")
        with open(preds_file, 'r') as f_pred:
            for line in f_pred:
                try:
                    pred_item = json.loads(line.strip())
                    # VQA API expects integer question_id
                    question_id = int(pred_item['id']) 
                    answer = pred_item['prediction']
                    vqa_preds_formatted.append({
                        'question_id': question_id,
                        'answer': answer
                    })
                except (json.JSONDecodeError, KeyError, ValueError) as e:
                    print(f"Warning: Skipping invalid prediction line: {line.strip()}. Error: {e}")
        print(f"Loaded {len(vqa_preds_formatted)} predictions for evaluation.")
        if not vqa_preds_formatted:
            print("Error: No valid predictions found in the file.")
            return results
        
        # --- Run VQA Evaluation --- 
        print("Initializing VQA evaluation...")
        vqa_ann = VQA(gt_file, ques_file) # Load annotations and questions
        vqa_pred = vqa_ann.loadRes(vqa_preds_formatted, ques_file) # Load predictions
        
        vqa_eval = VQAEval(vqa_ann, vqa_pred, n=2) # n=2 is standard for VQA
        
        print("Running evaluation...")
        vqa_eval.evaluate() # Perform evaluation
        
        print("Evaluation complete. Results:")
        vqa_eval.showEvals() # Print detailed results
        
        # Extract results into a dictionary
        results = {
            "overall": vqa_eval.accuracy['overall'],
            "yes/no": vqa_eval.accuracy['perAnswerType']['yes/no'],
            "number": vqa_eval.accuracy['perAnswerType']['number'],
            "other": vqa_eval.accuracy['perAnswerType']['other'],
        }
        print(f"Parsed VQA Accuracy: {results}")

    except FileNotFoundError as e:
         print(f"Error: Required file not found: {e}")
    except Exception as e:
        print(f"Error during VQA evaluation: {e}")
        import traceback
        traceback.print_exc()

    # Log metric to W&B if active
    if wandb.run is not None:
        wandb.log({"eval/vqa_score_overall": results.get("overall", 0.0)})
        wandb.log({"eval/vqa_score_yes_no": results.get("yes/no", 0.0)})
        wandb.log({"eval/vqa_score_number": results.get("number", 0.0)})
        wandb.log({"eval/vqa_score_other": results.get("other", 0.0)})
        
    return results

## Step 5.6: Implement ANLS Calculation for TextVQA/DocVQA

In [ ]:
#| export
def calculate_single_anls(prediction: str, ground_truths: List[str], threshold=0.5) -> float:
    """Calculates the Average Normalized Levenshtein Similarity (ANLS) for a single prediction.
    
    Based on the ANLS definition for ICDAR 2019 Robust Reading Challenge on Scene Text VQA.
    https://rrc.cvc.uab.es/?ch=11&com=tasks -> Task 3 -> Evaluation

    Args:
        prediction: The predicted answer string.
        ground_truths: A list of ground truth answer strings.
        threshold: The Normalized Levenshtein Distance threshold (default: 0.5).
                   If NLD > threshold, the similarity is 0.

    Returns:
        The ANLS score for this prediction (float between 0 and 1).
    """
    if Levenshtein is None:
        print("Warning: Levenshtein library not available. Returning 0 for ANLS.")
        return 0.0
    
    if not ground_truths: # Handle case with no ground truths
        return 0.0
    if not prediction: # Handle case with empty prediction
        # Similarity is 1 if any GT is also empty, 0 otherwise
        return 1.0 if any(not gt for gt in ground_truths) else 0.0
        
    max_similarity = 0.0
    prediction_lower = prediction.lower() # Typically case-insensitive

    for gt in ground_truths:
        gt_lower = gt.lower()
        if not gt_lower:
            # Similarity is 0 if prediction is non-empty and GT is empty
            similarity = 0.0
        else:
            distance = Levenshtein.distance(prediction_lower, gt_lower)
            max_len = max(len(prediction_lower), len(gt_lower))
            if max_len == 0: # Avoid division by zero if both strings are empty after lowercasing etc.
                 nld = 0.0
            else:
                 nld = distance / max_len # Normalized Levenshtein Distance

            if nld <= threshold:
                similarity = 1.0 - nld
            else:
                similarity = 0.0
        
        max_similarity = max(max_similarity, similarity)
        
    return max_similarity

#| export
def evaluate_textvqa(preds_file: Union[str, Path], 
                     gt_file: Union[str, Path], 
                     anls_threshold=0.5,
                     **kwargs):
    """Evaluates TextVQA/DocVQA predictions using the ANLS metric.

    Assumes standard TextVQA JSON format for ground truth and
    a JSON Lines file for predictions with 'id' (question_id) and 'prediction'.

    Args:
        preds_file: Path to the JSON Lines prediction file.
        gt_file: Path to the ground truth annotation file (e.g., TextVQA_0.5.1_val.json).
        anls_threshold: NLD threshold for ANLS calculation (default: 0.5).
        **kwargs: Additional arguments (unused).

    Returns:
        Dictionary containing ANLS score (e.g., {'anls': score}).
    """
    print(f"--- TextVQA/DocVQA ANLS Evaluation --- ")
    print(f"Predictions file: {preds_file}")
    print(f"Ground Truth file: {gt_file}")
    
    if Levenshtein is None:
        print("Error: `pip install python-Levenshtein` is required for ANLS evaluation.")
        return {"anls": 0.0}
        
    total_anls = 0.0
    count = 0
    results = {"anls": 0.0} # Default return value
    
    try:
        # Load ground truth
        with open(gt_file, 'r') as f_gt:
            ground_truth_data = json.load(f_gt)
            
        # Format ground truth into a dictionary {question_id: [answer1, answer2, ...]}
        # Adapting for TextVQA format which uses 'question_id' and 'answers'
        if 'data' in ground_truth_data:
            gt_dict = {item['question_id']: item['answers'] for item in ground_truth_data['data']}
        else:
             print(f"Warning: Unexpected GT file format in {gt_file}. Expected a 'data' key.")
             gt_dict = {}
             
        # Load predictions
        predictions = []
        with open(preds_file, 'r') as f_pred:
            for line in f_pred:
                try:
                    predictions.append(json.loads(line.strip()))
                except json.JSONDecodeError:
                    print(f"Warning: Skipping invalid JSON line in predictions file: {line.strip()}")
                    
        print(f"Loaded {len(gt_dict)} ground truth items and {len(predictions)} predictions.")
        
        # Calculate ANLS for each prediction
        for pred_item in tqdm(predictions, desc="Calculating ANLS"):
            pred_id = pred_item.get('id')
            prediction = pred_item.get('prediction', '')
            
            if pred_id is None:
                print(f"Warning: Prediction item missing 'id': {pred_item}")
                continue
                
            # Ensure ID matching (GT keys are strings in TextVQA JSON)
            gt_answers = gt_dict.get(str(pred_id))
            
            if gt_answers is None:
                # print(f"Warning: No ground truth found for prediction ID: {pred_id}")
                # Some benchmarks might have predictions for samples not in the target GT split
                continue # Skip if no ground truth for this prediction
            
            sample_anls = calculate_single_anls(prediction, gt_answers, threshold=anls_threshold)
            total_anls += sample_anls
            count += 1
            
        if count > 0:
            final_anls = total_anls / count
            results = {"anls": final_anls}
            print(f"Evaluated {count} samples.")
            print(f"Average Normalized Levenshtein Similarity (ANLS): {final_anls:.4f}")
        else:
             print("No matching predictions found for ground truth IDs. ANLS is 0.")
             results = {"anls": 0.0}

    except FileNotFoundError as e:
        print(f"Error: Required file not found: {e}")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
    except Exception as e:
        print(f"Error during ANLS calculation: {e}")
        import traceback
        traceback.print_exc()

    # Log metric to W&B if active
    if wandb.run is not None:
        wandb.log({"eval/anls": results.get("anls", 0.0)})
        
    return results

#### Test ANLS Calculation

In [ ]:
#| test
if Levenshtein:
    # Test cases for calculate_single_anls
    assert abs(calculate_single_anls("apple", ["apple"]) - 1.0) < 1e-6, "Test 1 Failed (Exact Match)"
    assert abs(calculate_single_anls("apple", ["aple"]) - (1 - 1/5)) < 1e-6, "Test 2 Failed (One deletion)"
    assert abs(calculate_single_anls("apple", ["banana"]) - 0.0) < 1e-6, "Test 3 Failed (High distance)"
    assert abs(calculate_single_anls("", ["apple"]) - 0.0) < 1e-6, "Test 4 Failed (Empty prediction)"
    assert abs(calculate_single_anls("apple", [""]) - 0.0) < 1e-6, "Test 5 Failed (Empty GT)"
    assert abs(calculate_single_anls("apple", ["APPLE"]) - 1.0) < 1e-6, "Test 6 Failed (Case difference)"
    assert abs(calculate_single_anls("apple", ["banana", "apple", "orange"]) - 1.0) < 1e-6, "Test 7 Failed (Multiple GT, one match)"
    # Test 8: distance("apple pie", "apple") = 4, max_len=9, nld=4/9=0.444. Since 0.444 <= 0.5, similarity = 1 - 0.444 = 0.555
    assert abs(calculate_single_anls("apple pie", ["apple"]) - (1 - 4/9)) < 1e-6, "Test 8 Failed (NLD < threshold)"
    print("ANLS calculation tests passed.")
else:
    # Print expected vs got if Levenshtein is not installed
    print(f"Test 1 (Exact Match): Expected 1.0, Got {calculate_single_anls('apple', ['apple'])} - Levenshtein not installed.")
    print(f"Test 2 (One deletion): Expected >0.8, Got {calculate_single_anls('apple', ['aple'])} - Levenshtein not installed.")
    print(f"Test 3 (High distance): Expected 0.0, Got {calculate_single_anls('apple', ['banana'])} - Levenshtein not installed.")
    print(f"Test 4 (Empty prediction): Expected 0.0, Got {calculate_single_anls('', ['apple'])} - Levenshtein not installed.")
    print(f"Test 5 (Empty GT): Expected 0.0, Got {calculate_single_anls('apple', [''])} - Levenshtein not installed.")
    print(f"Test 6 (Case difference): Expected 1.0, Got {calculate_single_anls('apple', ['APPLE'])} - Levenshtein not installed.")
    print(f"Test 7 (Multiple GT, one match): Expected 1.0, Got {calculate_single_anls('apple', ['banana', 'apple', 'orange'])} - Levenshtein not installed.")
    print(f"Test 8 (NLD < threshold): Expected ~0.56, Got {calculate_single_anls('apple pie', ['apple'])} - Levenshtein not installed.")

Test 1 (Exact Match): Expected 1.0, Got 0.0 - Levenshtein not installed.
Test 2 (One deletion): Expected >0.8, Got 0.0 - Levenshtein not installed.
Test 3 (High distance): Expected 0.0, Got 0.0 - Levenshtein not installed.
Test 4 (Empty prediction): Expected 0.0, Got 0.0 - Levenshtein not installed.
Test 5 (Empty GT): Expected 0.0, Got 0.0 - Levenshtein not installed.
Test 6 (Case difference): Expected 1.0, Got 0.0 - Levenshtein not installed.
Test 7 (Multiple GT, one match): Expected 1.0, Got 0.0 - Levenshtein not installed.
Test 8 (NLD < threshold): Expected ~0.56, Got 0.0 - Levenshtein not installed.


## Step 7.4: Custom Evaluation Set Evaluation

Define a function specifically for evaluating the custom dataset. This might involve custom metrics or qualitative analysis later. For now, we'll implement it using ANLS as a placeholder, assuming the custom set has a compatible format (questions/answers).

In [ ]:
#| export
def evaluate_custom_set(preds_file: Union[str, Path],
                        gt_file: Union[str, Path],
                        **kwargs):
    """Evaluates predictions on the custom evaluation set.
    
    Placeholder implementation. Currently uses ANLS assuming a TextVQA-like format.
    This should be adapted based on the specific metrics defined for the custom set.

    Args:
        preds_file: Path to the JSON Lines prediction file.
        gt_file: Path to the ground truth annotation file for the custom set.
        **kwargs: Additional arguments (e.g., ANLS threshold).

    Returns:
        Dictionary containing evaluation scores.
    """
    print(f"--- Custom Evaluation Set Evaluation --- ")
    print(f"Predictions file: {preds_file}")
    print(f"Ground Truth file: {gt_file}")

    # --- Placeholder: Use ANLS for now --- 
    # This assumes the custom eval set follows a format where ANLS is meaningful
    # (e.g., question_id, answers in GT; id, prediction in preds)
    # You might need to implement different logic or metrics here.
    if Levenshtein is None:
        print("Error: Levenshtein library not installed. Cannot calculate ANLS for custom set.")
        return {"custom_metric_placeholder": 0.0}
        
    anls_threshold = kwargs.get('anls_threshold', 0.5)
    results = evaluate_textvqa(preds_file, gt_file, anls_threshold=anls_threshold)
    
    # Rename the key for clarity and log to W&B
    custom_results = {f"custom_anls@{anls_threshold}": results.get("anls", 0.0)}
    print(f"Custom Set ANLS Results: {custom_results}")

    if wandb.run is not None:
        wandb.log({f"eval/custom_anls": custom_results.get(f"custom_anls@{anls_threshold}", 0.0)})
    
    # --- Add other custom metrics calculation here --- 
    # Example: Calculate accuracy if applicable
    # custom_results['accuracy'] = calculate_custom_accuracy(preds_file, gt_file)
    
    return custom_results

## Evaluation Script

In [ ]:
#| export
def run_evaluation(config_path: Union[str, Path], 
                   model_type: str = 'baseline', # Added model_type argument
                   model_checkpoint_path: Optional[Union[str, Path]] = None, 
                   dataset_name: str = 'vqav2_test', # Example default
                   ablation_mode:str = None, # Added ablation_mode argument
                   output_filename: Optional[str] = None,
                   **kwargs # Allow passing generation args like temp, top_p
                   ):
    """Runs the evaluation pipeline: load model, generate predictions, evaluate.
    Handles both baseline and adaptive model types.
    Calls the appropriate evaluation function based on dataset_name.
    
    Args:
        config_path: Path to the main configuration YAML.
        model_type: Type of model to evaluate ('baseline' or 'adaptive').
        model_checkpoint_path: Path to the specific model checkpoint base name/directory to load 
                               (e.g., '/path/to/output/models/stage2_llava_lora'). 
                               If None, uses the path derived from config['paths']['stage2_model_weights'].
                               Expects associated files like '_projector_final.pth' and '_lora_adapters'/'_full.pth'.
                               Filenames will be appended with _{model_type} (e.g., _baseline_projector_final.pth).
        dataset_name: Name of the dataset split to evaluate (e.g., 'vqav2_test', 'textvqa_val', 'custom_eval').
                      This key should exist in `config['paths']` pointing to dataset info.
        output_filename: Name for the prediction output file (defaults based on model/dataset).
        **kwargs: Additional arguments passed to `generate_predictions` (e.g., temperature, top_p).
    """
    print(f"--- Starting Evaluation Run ({model_type.capitalize()} Model) --- ")
    print(f"Config: {config_path}")
    print(f"Dataset: {dataset_name}")
    
    config = load_config(config_path)
    output_dir = Path(config['paths']['output_dir'])
    eval_output_dir = output_dir / 'eval_results' / f"{model_type}_{dataset_name}" # Add model type to output dir
    eval_output_dir.mkdir(parents=True, exist_ok=True)
    models_dir = output_dir / 'models' # Where models are saved

    ablation_tag = f"_abl-{ablation_mode}" if ablation_mode else ""
    print(f"Ablation mode specified: {ablation_mode if ablation_mode else 'None'}")
    
    learner = None
    run = None # Initialize wandb run object
    try:
        # --- Initialize W&B --- 
        if config.get('logging', {}).get('wandb', {}).get('enabled', False):
             model_id_for_run = Path(model_checkpoint_path if model_checkpoint_path else config['paths']['stage2_model_weights']).stem
             run_name = f"eval_{model_type}{ablation_tag}_{model_id_for_run}_{dataset_name}_{time.strftime('%Y%m%d_%H%M%S')}"
             run = init_wandb(config, job_type="evaluation", run_name=run_name)
        
        # 1. Determine Model Path Base Name
        if model_checkpoint_path is None:
            base_name_from_config = config['paths'].get('stage2_model_weights', f'stage2_llava_{model_type}')
            model_base_name = f"{base_name_from_config}"
            model_load_base = models_dir / model_base_name 
            print(f"Using model base name from config: {model_base_name}")
        else:
            model_checkpoint_path = Path(model_checkpoint_path)
            model_base_name = model_checkpoint_path.stem
            model_load_base = model_checkpoint_path 
            print(f"Using provided model base path: {model_load_base}")
        
        # Define model-type specific filename suffix
        model_suffix = f"_{model_type.lower()}"

        # 2. Load Model Components Manually
        print(f"Loading {model_type} model components for evaluation...")
        ModelClass = AdaptiveLLaVAModel if model_type == 'adaptive' else BaselineLLaVAModel
        # Ensure LoRA/Checkpointing settings from config are used during model init
        # (though weights will be overwritten)
        eval_config = copy.deepcopy(config)
        if 'model' not in eval_config: eval_config['model'] = {}
        # Disable training-specific memory optimizations for eval if needed
        eval_config['model']['use_activation_checkpointing'] = False

        model = ModelClass(config=eval_config) 
        
        # Load projector weights
        proj_weights_path = model_load_base.parent / (model_load_base.name + f"{model_suffix}{ablation_tag}_projector_final.pth")
        if proj_weights_path.exists():
            model.projector.load_state_dict(torch.load(proj_weights_path, map_location='cpu'))
            print(f"Loaded projector weights from {proj_weights_path}")
        else:
            print(f"Warning: Projector weights not found at {proj_weights_path}. Using initial weights.")
            
        # Load LoRA adapters
        use_lora_config = config.get('model', {}).get('peft', {}).get('use_lora', False)
        lora_adapter_path = model_load_base.parent / (model_load_base.name + f"{model_suffix}{ablation_tag}_lora_adapters")

        if use_lora_config:
            if lora_adapter_path.exists() and _peft_available and isinstance(model.language_model, PeftModel):
                print(f"Loading LoRA adapters from {lora_adapter_path}")
                try:
                    if hasattr(model.language_model, 'load_adapter'):
                         model.language_model.load_adapter(str(lora_adapter_path), adapter_name="default")
                         print("LoRA adapters loaded successfully.")
                    else:
                         print("Warning: Model's language_model does not have 'load_adapter' method.")
                except Exception as e:
                    print(f"Error loading LoRA adapters: {e}.")
            else:
                print(f"Warning: LoRA enabled but adapters not found at {lora_adapter_path} or PEFT issue.")
        else:
             print("LoRA not used.")
             
        # Load Patcher weights (if adaptive)
        if model_type == 'adaptive' and hasattr(model, 'patcher') and model.patcher is not None:
             patcher_params = list(model.patcher.parameters())
             # Check if patcher has parameters (even if not trained, might be loaded)
             if patcher_params: 
                  patcher_weights_path = model_load_base.parent / (model_load_base.name + f"{model_suffix}{ablation_tag}_patcher_final.pth")
                  if patcher_weights_path.exists():
                       model.patcher.load_state_dict(torch.load(patcher_weights_path, map_location='cpu'))
                       print(f"Loaded patcher weights from {patcher_weights_path}")
                  else:
                       print(f"Warning: Patcher weights file not found at {patcher_weights_path}")
             else:
                  print("Adaptive patcher exists but has no parameters.")
        
        # 3. Create Learner Shell
        dummy_dl = DataLoader(list(range(1)), bs=1)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        dummy_dls = DataLoaders(dummy_dl, dummy_dl)
        dummy_dls.device = device 
        learner = Learner(dls=dummy_dls, model=model, loss_func=lambda x,y: 0)
        
        global tokenizer 
        if tokenizer is None:
             raise RuntimeError("Tokenizer not loaded, cannot proceed with evaluation.")
        learner.tokenizer = tokenizer 
        print(f"Minimal Learner shell created. Model moved to {device}.")
        
        # 4. Load Test Data
        print(f"Loading test data for {dataset_name}...")
        if LLaVADataBlockStage2 is None:
             raise RuntimeError("LLaVADataBlockStage2 is not defined. Cannot load test data.")
        test_dl = get_test_dataloader(config, dataset_name, dblock=LLaVADataBlockStage2)
        if test_dl is None:
            raise RuntimeError(f"Failed to load test dataloader for {dataset_name}")

        # 5. Generate Predictions
        if output_filename is None:
            output_filename = f"preds_{model_base_name}{model_suffix}{ablation_tag}_{dataset_name}.jsonl"
        preds_file = eval_output_dir / output_filename
        
        gen_kwargs = {k: v for k, v in kwargs.items() if k in ['max_len', 'temperature', 'top_p']}
        
        generate_predictions(learner, test_dl, preds_file, **gen_kwargs)

        # 6. Run Evaluation Metrics
        print("Running evaluation metrics...")
        gt_config = config['paths'].get(dataset_name)
        if not gt_config or 'annotations' not in gt_config:
            print(f"Warning: Ground truth annotation path not found for '{dataset_name}' in config. Cannot run metrics.")
        else:
            gt_file_path = Path(config['paths']['data_base']) / gt_config['annotations']
            ques_file_path = None
            if 'questions' in gt_config:
                 ques_file_path = Path(config['paths']['data_base']) / gt_config['questions']
            elif 'vqav2' in dataset_name.lower(): 
                 ann_path_parts = list(gt_file_path.parts)
                 if 'annotations' in ann_path_parts[-1]:
                      ques_filename = gt_file_path.name.replace('annotations', 'questions').replace('v2_mscoco', 'v2_OpenEnded_mscoco')
                      ques_file_path_default = gt_file_path.parent / ques_filename
                      if ques_file_path_default.exists():
                           ques_file_path = ques_file_path_default
                           print(f"Inferred VQA questions file path: {ques_file_path}")
            
            if not gt_file_path.exists():
                 print(f"Warning: Ground truth file not found at {gt_file_path}. Cannot run metrics.")
            else:
                 # --- Select Evaluation Function Based on Dataset Name --- #
                 if 'vqav2' in dataset_name.lower():
                     if ques_file_path and ques_file_path.exists():
                         vqa_results = evaluate_vqa(preds_file, gt_file_path, ques_file=ques_file_path)
                         print(f"VQAv2 Results: {vqa_results}")
                     else:
                          print(f"Warning: VQA questions file not found or specified ({ques_file_path}). Cannot run VQA evaluation.")
                 elif 'textvqa' in dataset_name.lower():
                     anls_results = evaluate_textvqa(preds_file, gt_file_path)
                     print(f"TextVQA ANLS Results: {anls_results}")
                 elif 'custom_eval' == dataset_name.lower(): # Use exact match for custom
                     custom_results = evaluate_custom_set(preds_file, gt_file_path)
                     print(f"Custom Eval Set Results: {custom_results}")
                 # --- Add more evaluation cases here --- #
                 # elif 'docvqa' in dataset_name.lower():
                 #      anls_results = evaluate_textvqa(preds_file, gt_file_path) # Reuse ANLS
                 #      print(f"DocVQA ANLS Results: {anls_results}")
                 # elif 'chartqa' in dataset_name.lower():
                 #      # chartqa_results = evaluate_chartqa(preds_file, gt_file_path)
                 #      print("ChartQA evaluation not implemented yet.")
                 else:
                     print(f"No specific evaluation script configured for dataset: {dataset_name}")

    except Exception as e:
        print(f"An error occurred during evaluation: {e}")
        import traceback
        traceback.print_exc()
        if run: run.finish(exit_code=1)
    finally:
        # Clean up memory
        if learner is not None and hasattr(learner, 'model') and learner.model is not None:
            learner.model.to('cpu')
            del learner.model
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            print("Cleaned up model memory.")
        # Finish W&B run if active and not already finished due to error
        if run and wandb.run and wandb.run.id == run.id:
             run.finish()
             print("Finished W&B run.")
            
    print(f"--- Evaluation Run Complete ({model_type.capitalize()} Model) --- ")

In [ ]:
#| export
# Command-line execution block
if __name__ == "__main__" and "get_ipython" not in locals():
    parser = argparse.ArgumentParser(description="Run LLaVA Model Evaluation")
    parser.add_argument("--config", type=str, default="configs/config.yaml", 
                        help="Path to the configuration YAML file (relative to project root or absolute).")
    parser.add_argument("--model_type", type=str, default="baseline", choices=['baseline', 'adaptive'],
                        help="Choose model type to evaluate: 'baseline' or 'adaptive'.")
    parser.add_argument("--model_checkpoint", type=str, default=None,
                        help="Path to the model checkpoint base name/directory (e.g., output/models/stage2_llava_lora). Optional. If provided, _{model_type} will be appended for weight loading.")
    parser.add_argument("--dataset", type=str, required=True,
                        help="Name of the dataset to evaluate (key in config['paths'], e.g., vqav2_test, textvqa_val, custom_eval).")
    parser.add_argument("--output_filename", type=str, default=None,
                        help="Optional custom name for the predictions output file.")
    # Add generation arguments
    parser.add_argument("--max_len", type=int, default=200, help="Max new tokens for generation.")
    parser.add_argument("--temperature", type=float, default=0.2, help="Generation temperature (0 for greedy).")
    parser.add_argument("--top_p", type=float, default=None, help="Nucleus sampling top_p (e.g., 0.9).")
    parser.add_argument("--ablation", type=str, default=None, choices=[None, 'baseline'], # Add ablation choices
                    help="Optional ablation mode being evaluated (e.g., 'baseline'). Affects weight loading path.")

    args = parser.parse_args()

    # Resolve config path relative to the determined project root
    config_file_path = project_root / args.config
    model_ckpt_path = Path(args.model_checkpoint) if args.model_checkpoint else None
    if model_ckpt_path and not model_ckpt_path.is_absolute(): # Resolve relative checkpoint path w.r.t project root
        model_ckpt_path = project_root / model_ckpt_path
    
    if not config_file_path.is_file():
        print(f"Error: Config file not found at {config_file_path}")
        sys.exit(1)
    
    # Pass generation args to run_evaluation
    gen_args = {
        "max_len": args.max_len,
        "temperature": args.temperature,
        "top_p": args.top_p
    }

    try:
        run_evaluation(
            config_path=config_file_path,
            model_type=args.model_type, # Pass model type
            model_checkpoint_path=model_ckpt_path,
            dataset_name=args.dataset,
            ablation_mode=args.ablation,
            output_filename=args.output_filename,
            **gen_args
        )
    except NotImplementedError as e:
         print(f"Exiting due to NotImplementedError: {e}")
         sys.exit(0)
    except Exception as e:
        print(f"Evaluation script failed: {e}")
        import traceback
        traceback.print_exc()
        sys.exit(1)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()